In [ ]:
!pip install unicodedata
!pip install transliterate
!pip install transformers
!pip install transformers[torch]
!pip install accelerate -U
!pip install datasets
!pip install evaluate
from datasets import load_dataset
from transformers import pipeline, AutoTokenizer, DataCollatorForLanguageModeling, AutoModelForCausalLM, TrainingArguments, Trainer, TextDataset
import math
import tensorflow as tf
import torch
from torch.utils.data import DataLoader, Dataset

ERROR: Could not find a version that satisfies the requirement unicodedata (from versions: none)
ERROR: No matching distribution found for unicodedata


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Loading the text generation model

In [ ]:
model = AutoModelForCausalLM.from_pretrained("distilgpt2", from_tf=True)

All TF 2.0 model weights were used when initializing GPT2LMHeadModel.

Some weights of GPT2LMHeadModel were not initialized from the TF 2.0 model and are newly initialized: ['lm_head.weight', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model_save_name = 'urdu-textgen5.pt'
path = F"/content/drive/My Drive/deep/urdu-textgen5.pt"
model.load_state_dict(torch.load(path))
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
if tokenizer.pad_token is None:
  tokenizer.pad_token = tokenizer.eos_token

# Function to generate text given a word

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained("hadidev/gpt2-urdu-smallest")
# model = AutoModelForCausalLM.from_pretrained("hadidev/gpt2-urdu-smallest", from_tf=True)

In [ ]:
def gen_word(prompt):
  inputs = tokenizer(prompt, return_tensors="pt").input_ids
  outputs = model.generate(inputs, max_new_tokens=10, do_sample=True, top_k=50, top_p=0.95)
  return tokenizer.batch_decode(outputs, skip_special_tokens=True)

gen_word("اس کے بارے میں بات")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['اس کے بارے میں بات ی   ب ی']

# Converting the generated word into Baudot code

In [ ]:
import numpy as np
class Stegonography:
    def __init__(self):
        self.keywords = list()
        self.ungrouped_keys = list()
        self.groups = dict()
        self.baudot_code = dict()
        self.urdu_to_baudot_mapping = {
                                        'ا': '000000',
                                        'ب': '000001',
                                        'پ': '000010',
                                        'ت': '000011',
                                        'ٹ': '000100',
                                        'ث': '000101',
                                        'ج': '000110',
                                        'چ': '000111',
                                        'ح': '001000',
                                        'خ': '001001',
                                        'د': '001010',
                                        'ڈ': '001011',
                                        'ذ': '001100',
                                        'ر': '001101',
                                        'ڑ': '001110',
                                        'ز': '001111',
                                        'ژ': '010000',
                                        'س': '010001',
                                        'ش': '010010',
                                        'ص': '010011',
                                        'ض': '010100',
                                        'ط': '010101',
                                        'ظ': '010110',
                                        'ع': '010111',
                                        'غ': '011000',
                                        'ف': '011001',
                                        'ق': '011010',
                                        'ک': '011011',
                                        'گ': '011100',
                                        'ل': '011101',
                                        'م': '011110',
                                        'ن': '011111',
                                        'ں': '100000',
                                        'ھ': '100001',
                                        'و': '100010',
                                        'ء': '100011',
                                        'ی': '100100',
                                        'ے': '100101',
                                        'ئ': '100110',
                                        'ہ': '100111',
                                        'ۓ': '101001',
                                        'آ': '101010',
                                        'ؤ': '101011'
                                    }



    def set_keyword(self, keyword):
        self.keywords.append(keyword)
        self.ungrouped_keys.append(keyword)
    def get_group(self):
        for key in self.ungrouped_keys:
            # index = np.arange(len(key))
            # letters = np.array([*key])
            # letters = np.unique(letters)
            # np.random.shuffle(letters)
            # group_0 = letters[:len(letters)//2].tolist()
            # group_1 = letters[len(letters)//2:].tolist()
            # self.groups.update({key:[group_0,group_1 ]})
            # self.ungrouped_keys.remove(key)


            letters = np.array([*'کیاآپکومیںکچھ'])
            letters = np.unique(letters)
            np.random.shuffle(letters)
            group_0 = letters[:len(letters)//2].tolist()
            group_1 = letters[len(letters)//2:].tolist()
            self.groups.update({key:[group_0,group_1 ]})
            self.ungrouped_keys.remove(key)

    def encode(self):
        for keyword in self.keywords:
            print(keyword)
            code = ''
            letters = [*keyword]
            for letter in letters:

                if letter in list(self.urdu_to_baudot_mapping.keys()):
                    print(f'The code is \n{code} and The letter = {letter}')

                    code += self.urdu_to_baudot_mapping[letter]
                else:
                    print(f'Please use a letter recognized by the dictionary, {letter}')
            code = code
            self.baudot_code[keyword] = (code)
        return

    def decode(self, baudot ):
        values = list(self.urdu_to_baudot_mapping.values())
        keys = list(self.urdu_to_baudot_mapping.keys())
        decoded = []
        for i in range(0, int(len(baudot)),6):
            code = (baudot[i:i+6])


            if code in values:
                j = values.index(code)
                decoded.append(keys[j])
                print(f'the code is {code} and the letter is {keys[j]}')
        print(decoded)

    def split_urdu_word(self, word):
      letters = [letter for letter in word]
      return letters

    def check_word(self, testword, baudot_code, grp_1, grp_0):
        letters = self.split_urdu_word(testword)
        print(letters, testword)
        # print(tst_word)
        # print(letters)
        lgth = 0
        error = 0
        bdt_code = list(baudot_code)
        var = bdt_code.copy()
        while(True):
          if (lgth > len(letters)-1) :
            break
          chr = letters[lgth]
          # print(chr, lgth, letters)
          if (chr in grp_0 or chr in grp_1):
            # print(var[0], chr , grp_1 )
            if (len(var) <=0) :
              print("Need to regenerate")
              error = 1
              break

            elif ((chr in grp_0) and (var[0]!= '0')):
              print("Need to regenerate")
              error = 1
              break
            elif ((chr in grp_1) and (var[0]!= '1')):
              print("Need to regenerate")
              error = 1
              break
            else:
              var.pop(0)
          #loop over the word
          lgth+=1

        if(error == 0):
            print(f'The remaining baudot code is \n{var}')
            return True, "".join(var)
        return False,  "".join(bdt_code)

    def remove_spaces(self, output) -> str:
        lst = output[0].split('   ')[1:]
        sentence = ''
        for elm in lst:
            for letter in elm:
                if letter != ' ':
                    sentence += letter
                else:
                    continue
            sentence += ' '
        return sentence
    def gen_word(self,prompt):
        inputs = tokenizer(prompt, return_tensors="pt").input_ids
        outputs = model.generate(inputs, pad_token_id=tokenizer.pad_token_id, max_new_tokens=10, do_sample=True, top_k=50, top_p=0.95)
        return tokenizer.batch_decode(outputs, skip_special_tokens=True)
    def stegger(self, secret_msg, initial_prompt):
        self.set_keyword(secret_msg)
        self.get_group()
        self.encode()
        baudot_code = self.baudot_code[secret_msg]
        groups = self.groups[secret_msg]
        grp_0 = groups[0]
        grp_1 = groups[1]
        ''' loop variables initialize'''
        generated = self.gen_word(initial_prompt)
        test_word = self.remove_spaces(generated)
        remaining_bc = baudot_code
        new_prompt = initial_prompt
        self.cover = ''
        j = 0
        first_check = False
        while True:
            j += 1

            if not j%20:
                print(j, end='')
            check, remaining_bc = self.check_word(test_word, remaining_bc, grp_1, grp_0)
            if check:
                new_prompt += test_word
                if not first_check:
                    self.cover = test_word
                else:
                    self.cover += test_word
                first_check = True

                generated = self.gen_word(self.cover)
                test_word = self.remove_spaces(generated)
            else:
                generated = self.gen_word(self.cover)
                test_word = self.remove_spaces(generated)

            print(f'{self.cover}')
            if len(remaining_bc) == 0:
                print('Message has been hidden')
                break
        # return cover

In [ ]:
hider = Stegonography()
prompt ="میں"
secret = 'حبیب'
hider.stegger(secret, prompt)

حبیب
The code is 
 and The letter = ح
The code is 
001000 and The letter = ب
The code is 
001000000001 and The letter = ی
The code is 
001000000001100100 and The letter = ب
['ا', 'س', 'ے', ' '] اسے 
The remaining baudot code is 
['0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '1']
اسے 
['ک', 'ر', 'ت', 'ا', ' '] کرتا 
Need to regenerate
اسے 
['ک', 'و', ' ', 'گ', ' '] کو گ 
Need to regenerate
اسے 
['م', 'ی', 'ٹ', 'ا', ' '] میٹا 
Need to regenerate
اسے 
['ک', 'ی', 'و', 'ن', ' '] کیون 
Need to regenerate
اسے 
['چ', 'ھ', 'و', 'ٹ', ' '] چھوٹ 
The remaining baudot code is 
['0', '0', '0', '0', '0', '0', '0', '1', '1', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '1']
اسے چھوٹ 
['ک', 'ے', ' ', ' '] کے  
Need to regenerate
اسے چھوٹ 
['م', ' '] م 
Need to regenerate
اسے چھوٹ 
['ک', 'ر', '�', ' '] کر� 
Need to regenerate
اسے چھوٹ 
['ش', 'ی', '�', ' '] شی� 
Need to regenerate
اسے چھوٹ 
['ہ', ' '] ہ 
The remaining baudot code is 
[

In [ ]:
xx = ''
for elm in secret:
  x = hider.urdu_to_baudot_mapping[elm]
  print(f'{elm} = {hider.urdu_to_baudot_mapping[elm]}')
  xx += x
print(f'The complete binary representation of the word {secret} is:\n{xx}')

ح = 001000
ب = 000001
ی = 100100
ب = 000001
The complete binary representation of the word حبیب is:
001000000001100100000001


In [ ]:
string = hider.cover
xxx = tokenizer(string, return_tensors="pt").input_ids
print(string)
print(hider.groups[secret][1])
print(hider.baudot_code)
code = ''
group_0 = hider.groups[secret][0]
group_1 = hider.groups[secret][1]
for letter in string:
    if letter in group_0:
        code += '0'
    elif letter in group_1:
        code += '1'
    else:
        continue

print(code)
# i = 0
# for elm in string:
#     print(elm, i)
#     i += 1

اسے چھوٹ ہ ا� ہے۔ جنہو  آف در ہ  سے ہ واقعت خر� � اداک بھ ہر ع اور ب� کے � ہدا� ڈ � زندگ  آنے  اور ا� افر � جو س کے 
['ھ', 'ک', 'ی', 'م', 'پ']
{'حبیب': '001000000001100100000001'}
001000000001100100000001


In [ ]:
print(group_0)
print(group_1)

['ں', 'آ', 'چ', 'و', 'ا']
['ھ', 'ک', 'ی', 'م', 'پ']


In [ ]:
hider.decode(code)
hider.encode()

the code is 001000 and the letter is ح
the code is 000001 and the letter is ب
the code is 100100 and the letter is ی
the code is 000001 and the letter is ب
['ح', 'ب', 'ی', 'ب']
حبیب
The code is 
 and The letter = ح
The code is 
001000 and The letter = ب
The code is 
001000000001 and The letter = ی
The code is 
001000000001100100 and The letter = ب


In [ ]:
print('The secret message:', secret)
print()
print('The key used is: ')
print('group_0:', group_0)
print('group_1:', group_1)
print('The algorithm converts the secret message into the following: ')
hider.encode()
print()
print('The language model produces the following cover text: ')
print(hider.cover)
print()
print('The secret message extracted by the algorithm: ')
hider.decode(code)

The secret message: حبیب

The key used is: 
group_0: ['ں', 'آ', 'چ', 'و', 'ا']
group_1: ['ھ', 'ک', 'ی', 'م', 'پ']
The algorithm converts the secret message into the following: 
حبیب
The code is 
 and The letter = ح
The code is 
001000 and The letter = ب
The code is 
001000000001 and The letter = ی
The code is 
001000000001100100 and The letter = ب

The language model produces the following cover text: 
اسے چھوٹ ہ ا� ہے۔ جنہو  آف در ہ  سے ہ واقعت خر� � اداک بھ ہر ع اور ب� کے � ہدا� ڈ � زندگ  آنے  اور ا� افر � جو س کے 

The secret message extracted by the algorithm: 
the code is 001000 and the letter is ح
the code is 000001 and the letter is ب
the code is 100100 and the letter is ی
the code is 000001 and the letter is ب
['ح', 'ب', 'ی', 'ب']
